<a href="https://colab.research.google.com/github/arjunverma2004/CampusX-100-Days-of-Deep-Learning/blob/main/V50_data_augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# from keras.preprocessing import image
# from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array


In [2]:
img = load_img('/content/dog.jpg', target_size=(200, 200))

In [ ]:
type(img)

PIL.Image.Image

In [3]:
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)

In [4]:
img = img_to_array(img)

In [5]:
#To change the shape
input_batch = img.reshape((1,) + img.shape)

In [6]:
input_batch.shape

(1, 200, 200, 3)

In [7]:
import os
os.mkdir('preview')

In [8]:
i=0

for output in datagen.flow(input_batch, batch_size=1, save_to_dir='preview'):
  i+=1
  if i>10:
    break

In [9]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("salader/dogs-vs-cats")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/dogs-vs-cats


###Creating Model

In [14]:
import shutil
import os

# Define source and destination paths
source_folder = "/kaggle/input/dogs-vs-cats"
destination_folder = "content"

try:
    # Copy the entire folder
    shutil.copytree(source_folder, destination_folder)
    print(f"Folder '{source_folder}' copied successfully to '{destination_folder}'.")
except FileExistsError:
    print(f"Destination folder '{destination_folder}' already exists. Please remove it or choose a different name.")
except Exception as e:
    print(f"An error occurred: {e}")

Folder '/kaggle/input/dogs-vs-cats' copied successfully to 'content'.


In [20]:
os.mkdir('dogs_vs_cats')

In [22]:
os.mkdir('dogs_vs_cats2/valid')

In [32]:
batch_size = 16
#this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
  rescale=1./255,
  shear_range=0.2,
  zoom_range=0.2,
  horizontal_flip=True)

#this is the augmentation configuration we will use for testing:
#only rescaling

test_datagen = ImageDataGenerator(rescale=1./255)



#this is a generator that will read pictures found in
#subfolers of 'data/train', and indefinitely generate
#batches of augmented image data

train_generator = train_datagen.flow_from_directory(
    '/content/content/dogs_vs_cats/train', # this is the target directory
    target_size=(150, 150), # all images will be resized to 150x150
    batch_size=batch_size,
    class_mode='binary') # since we use binary_crossentropy loss, we need binary Labels
#this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
    '/content/content/dogs_vs_cats/test',
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='binary')

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [33]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization

In [40]:
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2), strides=2, padding='valid'))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2), strides=2, padding='valid'))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2), strides=2, padding='valid'))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1, activation='sigmoid'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [41]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 148, 148, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 72, 72, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 34, 34, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 36992)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │     4,735,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,837,569 (18.45 MB)

 Trainable params: 4,837,121 (18.45 MB)

 Non-trainable params: 448 (1.75 KB)

In [43]:
model.compile(optimizer='adam',loss='binary_crossentropy', metrics=['accuracy'])

In [44]:
model.fit(
    train_generator,
    steps_per_epoch=2000 // batch_size,
    epochs=50,
    validation_data=validation_generator,
    validation_steps=800 // batch_size)

Epoch 1/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step - accuracy: 0.5280 - loss: 2.3247

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


125/125 ━━━━━━━━━━━━━━━━━━━━ 23s 116ms/step - accuracy: 0.5281 - loss: 2.3217 - val_accuracy: 0.5238 - val_loss: 0.8262
Epoch 2/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 20s 161ms/step - accuracy: 0.5630 - loss: 1.1639 - val_accuracy: 0.4950 - val_loss: 0.9030
Epoch 3/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 20s 163ms/step - accuracy: 0.5661 - loss: 0.8642 - val_accuracy: 0.5587 - val_loss: 0.9971
Epoch 4/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 20s 165ms/step - accuracy: 0.5822 - loss: 0.7091 - val_accuracy: 0.5650 - val_loss: 0.7882
Epoch 5/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 20s 165ms/step - accuracy: 0.5954 - loss: 0.7041 - val_accuracy: 0.5750 - val_loss: 0.7518
Epoch 6/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 11s 91ms/step - accuracy: 0.5985 - loss: 0.6698 - val_accuracy: 0.6400 - val_loss: 0.6207
Epoch 7/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 20s 160ms/step - accuracy: 0.6441 - loss: 0.6295 - val_accuracy: 0.6000 - val_loss: 0.6435
Epoch 8/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 8s 67ms/step - accuracy: 0.6410 - loss: 0.6439 - val_ac

KeyboardInterrupt: 